# La parallélisation, une tache plus ou moins complexe

La parallélisation n'est pas chose aisé. Le meilleur des cas est lorsque l'on en a pas besoin. Malgré tout, il est parfois nécessaire d'accélerer un peu les calculs.

Si s'en passer n'est pas possible, le meilleur cas de figure est lorsque la tache à effectuer consiste en N appels complètement indépendant d'une même fonction. Ce cas de figure est dit *embarrassingly parallel*. On verra très rapidement l'utilisation de `joblib` dans ce cas de figure.

Imaginons que l'on ai besoin d'utiliser une fonction lente `slow_square`, qui, donnée un nombre, renverra son carré :

In [1]:
import time
import numpy as np

In [8]:
def slow_square(a):
    print(f"squaring {a:g}...")
    time.sleep(1)
    squared_a = a ** 2
    print(f"{a:g} squared! result: {squared_a:g}")
    return squared_a

S'il est nécessaire d'appliquer cette fonction à un grand nombre d'entrée, le temps deviendra vite prohibitif.

In [7]:
%%time
squared = [slow_square(a) for a in np.linspace(0, 10, 4)]

squaring 0...
0 squared! result: 0
squaring 3.33333...
3.33333 squared! result: 11.1111
squaring 6.66667...


KeyboardInterrupt: 

L'appliquation d'une même fonction à une collection d'argument s'appel un `map`. Il est possible de faire la même chose que pour la cellule précédente avec :

In [9]:
squared = list(map(slow_square, np.linspace(0, 10, 4)))

squaring 0...
0 squared! result: 0
squaring 3.33333...
3.33333 squared! result: 11.1111
squaring 6.66667...
6.66667 squared! result: 44.4444
squaring 10...
10 squared! result: 100


En utilisant multiprocessing, on va pouvoir facilement paralléliser ce genre d'opération

In [10]:
import multiprocessing as mp

In [13]:
from joblib import delayed, Parallel, parallel_backend

In [20]:
with parallel_backend("loky", n_jobs=-1):
    futures = [delayed(slow_square)(a) for a in np.linspace(0, 10, 4)]
    print(futures)
    result = Parallel()(futures)

[(<function slow_square at 0x7fa54fe5d830>, (0.0,), {}), (<function slow_square at 0x7fa54fe5d830>, (3.3333333333333335,), {}), (<function slow_square at 0x7fa54fe5d830>, (6.666666666666667,), {}), (<function slow_square at 0x7fa54fe5d830>, (10.0,), {})]
squaring 0...
0 squared! result: 0
squaring 3.33333...
3.33333 squared! result: 11.1111
squaring 6.66667...
6.66667 squared! result: 44.4444
squaring 10...
10 squared! result: 100


In [16]:
result

[0.0, 11.111111111111112, 44.44444444444445, 100.0]

## Le cas d'Energy Plus

Il va falloir se méfier un petit peu plus : le logicier écrivant un grand nombre de fichiers intermédiaires, il va falloir travailler avec autant de dossier que de processus pour éviter de corrompre les fichiers de résultat.

In [21]:
from path import Path
import plumbum

In [22]:
eplus_bin = Path('/home/nicolas/.local/share/energy_plus_wrapper/EnergyPlus-8-7-0/energyplus')
idf_file = Path("../../data/eplus_data/in_8-7-0.idf")
epw_file = Path("../../data/eplus_data/in.epw")
eplus = plumbum.local[eplus_bin]

In [23]:
def launch_eplus_sim(parameters):
    idf_file, epw_file, out_dir = parameters
    eplus("-w", epw_file, "-d", out_dir, idf_file)

In [30]:
all_parameters = [(idf_file, epw_file, f"output_eplus_parallel/run_{i}")
                  for i in range(4)]
all_parameters

[(Path('../../data/eplus_data/in_8-7-0.idf'),
  Path('../../data/eplus_data/in.epw'),
  'output_eplus_parallel/run_0'),
 (Path('../../data/eplus_data/in_8-7-0.idf'),
  Path('../../data/eplus_data/in.epw'),
  'output_eplus_parallel/run_1'),
 (Path('../../data/eplus_data/in_8-7-0.idf'),
  Path('../../data/eplus_data/in.epw'),
  'output_eplus_parallel/run_2'),
 (Path('../../data/eplus_data/in_8-7-0.idf'),
  Path('../../data/eplus_data/in.epw'),
  'output_eplus_parallel/run_3')]

In [25]:
with parallel_backend("loky", n_jobs=-1):
    futures = [delayed(launch_eplus_sim)(parameters)
               for parameters in all_parameters]
    print(futures)
    result = Parallel()(futures)

[(<function launch_eplus_sim at 0x7fa54da05dd0>, ((Path('../../data/eplus_data/in_8-7-0.idf'), Path('../../data/eplus_data/in.epw'), 'output_eplus_parallel/run_0'),), {}), (<function launch_eplus_sim at 0x7fa54da05dd0>, ((Path('../../data/eplus_data/in_8-7-0.idf'), Path('../../data/eplus_data/in.epw'), 'output_eplus_parallel/run_1'),), {}), (<function launch_eplus_sim at 0x7fa54da05dd0>, ((Path('../../data/eplus_data/in_8-7-0.idf'), Path('../../data/eplus_data/in.epw'), 'output_eplus_parallel/run_2'),), {}), (<function launch_eplus_sim at 0x7fa54da05dd0>, ((Path('../../data/eplus_data/in_8-7-0.idf'), Path('../../data/eplus_data/in.epw'), 'output_eplus_parallel/run_3'),), {})]
